<h2>[ K-Fold_Cross_Validation ] </h2>

<h3>1. Dataset의 사용 Option들</h3>

<h4>Option1 : Use all available data for training and test on same dataset</h4>
<ul>
  <li>training에 data 100%를 사용하고 test에 동일하게 100%를 사용하는 방법</li>
  <li>100개의 문제를 사람이 풀고 다시 100개의 문제를 풀어 평가하기</li>
  <li>위의 예시에서 문제를 이미 확인하도 다시 푸는 것이므로 문제가 발생</li>
</ul>


<h4>Option2 : Split available dataset into training and test sets</h4>
<img src="../Image/CH12/Kfold_1.JPG" width="600" height="300" title=""/>
<ul>
  <li>train_test_split 사용</li>
  <li>70개의 문제를 주고 남은 30개의 문제로 평가히기</li>
  <li>문제 : 70개의 문제 종류와 나눈 30개의 문제 종류가 완전히 다를 경우가 있음</li>
</ul>


<h4>Option3 : K fold cross validation</h4>
<img src="../Image/CH12/Kfold_2.JPG" width="600" height="300" title=""/>
<ul>
  <li>먼제 dataset을 몇개의 fold로 만든다</li>
  <li>생성한 fold 중에서 한개만을 test set으로 하고 나머지를 train set으로 한다</li>
  <li>5 folds이라면 5번의 score가 나오는데 각 score의 평균을 구한다.</li>
  <li>train set에 model이 overfitting 된 경우를 방지하기 위해 교차검증을 사용한다 => K-fold</li>
</ul>

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()

---

<h4>train_test_split을 하는 경우 아래 세가지 model의 결과가 계속 변경된다</h4>

<h5>1. LogitsticRegression</h5>

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size = 0.3)

In [63]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\jack0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9685185185185186

<h5>2. SVM</h5>

In [64]:
svm = SVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.9833333333333333

<h5>3. RandomForestClassifier</h5>

In [65]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9722222222222222

---

<h4>1. K-fold</h4>

In [66]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3) # n_splits : fold를 몇 개로 설정할 것인지
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [67]:
# 3 folds일 떄, 어떻게 쪼개는지 확인하기

for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


<h5>모델을 훈련하고 score를 출력하는 함수 만들기</h5>

In [68]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [69]:
get_score(LogisticRegression(), X_train, X_test, y_train, y_test)

C:\Users\jack0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9685185185185186

In [70]:
get_score(SVC(), X_train, X_test, y_train, y_test)

0.9833333333333333

<h4>2. StratifiedKFold</h4>
<h5>K-fold에서 fold를 나누는데 한쪽으로만 특정 종류가 치우치는 경우를 방지하기 위해 더욱 균일하게 분할하는 StratifiedKFold를 사용하는게 더 정확함</h5>
<ul>
    <li>K-fold : dataset을 일정한 간격으로 잘라서 사용하므로 train set에 특정 종류의 data가 포함되지 않을 수 있음</li>
    <li>StratifiedKFold : target에서 속성값의 개수를 동일하게 가져가 데이터 쏠림 현상을 방지(K-fold의 단점 보완)</li>
    <li>일반적으로 분류에 StratifiedKFold를, 회귀에 기본 K-fold를 사용한다.</li>
</ul>

In [71]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=3)

In [72]:
scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    # scores_logistic.append(get_score(LogisticRegression(), X_train, X_test, y_train, y_test))  
    # scores_svm.append(get_score(SVC(), X_train, X_test, y_train, y_test))
    # scores_rf.append(get_score(RandomForestClassifier(), X_train, X_test, y_train, y_test))
    
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [73]:
scores_logistic

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]

In [74]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [75]:
scores_rf

[0.9332220367278798, 0.9415692821368948, 0.9148580968280468]

---

<h4>위의 복잡하고 긴 코드를 sklearn의 cross_val_score로 간단하게 가능함</h4>
<h5>cross_val_score(estimator, X,Y, scoring, cv)</h5>
<ul>
    <li>estimator : 분류, 회귀인지 구분 인자</li>
    <li>X : feature data set</li>
    <li>Y : label data set</li>
    <li>scoring : 예측 성능 평가 지표</li>
    <li>cv : 교차 검증 fold 수</li>
    <li>https://davinci-ai.tistory.com/18</li>
</ul>

In [77]:
from sklearn.model_selection import cross_val_score

cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), digits.data, digits.target,cv=3)

array([0.89482471, 0.95325543, 0.90984975])

In [78]:
cross_val_score(SVC(gamma='auto'), digits.data, digits.target,cv=3)

array([0.38063439, 0.41068447, 0.51252087])

In [79]:
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target,cv=3)

array([0.94490818, 0.94156928, 0.91986644])

---
참고 : https://homeproject.tistory.com/6